In [1]:
import os
import time
import numpy as np

import hoomd
import gsd.hoomd
import coxeter

from meta_alchemy import MetaAlchemUpdater

#### initializing snapshot

In [2]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=9520)
sim.create_state_from_gsd(filename='./DATA/compress.gsd', frame=-1)

#### initializing integrator

In [3]:
alpha_init=1.0
family323 = coxeter.families.Family323Plus()
particle = family323.get_shape(a=alpha_init, c=0.2*alpha_init+0.8)
verts = particle.vertices/particle.volume**(1/3)

mc = hoomd.hpmc.integrate.ConvexPolyhedron()
mc.shape["A"] = dict(vertices=verts)

#### harmonic constraint

In [4]:
target = gsd.hoomd.open('./DATA/lattice.gsd', 'r')[-1]
pos = target.particles.position
ori = target.particles.orientation
box = target.configuration.box

last_frame = sim._state.get_snapshot()
last_box = last_frame.configuration.box

rescale = last_box[0]/box[0]
pos = rescale*pos

k_init=20
k_second=0

k_rot_init=10
k_rot_second=0

t_ramp=int(1e5)
k_trans = hoomd.variant.Ramp(A=k_init, 
                              B=k_second,
                              t_start=sim.timestep, 
                              t_ramp=t_ramp)
k_rot = hoomd.variant.Ramp(A=k_rot_init, 
                            B=k_rot_second,
                            t_start=sim.timestep, 
                            t_ramp=t_ramp)

harmonic = hoomd.hpmc.external.field.Harmonic(reference_positions=pos,
                                              reference_orientations=ori,
                                              k_translational=k_trans,
                                              k_rotational=k_rot,
                                              symmetries=ori)
mc.external_potential=harmonic

#### alchemical updater

In [5]:
rng = np.random.default_rng(1234) # random generator
alchemupdater = hoomd.update.CustomUpdater(action=MetaAlchemUpdater(stepsize=0.005, 
                                                                    rng=rng,
                                                                    alpha_init=alpha_init), trigger=1)
metad_seed=59221
alchemupdater.set_metad_param(rng=np.random.default_rng(metad_seed),
                              h0=0.1,
                              sigma=0.02,
                              T=1.0,
                              dT=8.0,
                              stride=100,
                              calc_ebetac=False)

#### write

In [6]:
logger = hoomd.logging.Logger()
logger.add(mc, quantities=['type_shapes'])
logger.add(alchemupdater, quantities=['alpha', 'vbias'])

gsd_writer = hoomd.write.GSD(filename='./DATA/trajectory.gsd',
                             trigger=hoomd.trigger.Periodic(100),
                             mode='wb',
                             filter=hoomd.filter.All())
gsd_writer.logger=logger

#### attaching operations

In [7]:
sim.operations.writers.append(gsd_writer)
sim.operations.integrator = mc
sim.operations.updaters.append(alchemupdater)

#### run simulation

In [8]:
start = time.time()

sim.run(1e4)

print('Time elapsed', time.time()-start)

/Users/lina492375qw1188/opt/miniconda3/envs/alchemy/lib/python3.11/site-packages/hoomd/data/collections.py:241: IsolationWarning: The data structure is removed from its original data source, and updates will no longer modify the previously composing object. Call obj.to_base() to remove this warning.
  warnings.warn(hoomd.error.IsolationWarning())


Time elapsed 193.6946940422058


#### check overlaps

In [9]:
print(mc.overlaps, alchemupdater.alchem_moves)

0 (3937, 6063)


In [10]:
gsd_writer.flush()